In [1]:
import os
import torch
import torchaudio
import scipy.io.wavfile as wf
import numpy as np
import base64
import hashlib
import json 
import time
import uuid
import requests
import scipy
import random
#wav2mel = torch.jit.load("wav2mel.pt")
dvector = torch.jit.load("dvector-step250000.pt").eval()
n_fft = 1024
win_length = None

hop_length = 128
n_mels = 40
sample_rate = 16000

def dvector_extraction(wav_tensor1):
    #transform = torchaudio.transforms.MFCC(sample_rate = sample_rate,n_mfcc = 40, melkwargs={"n_fft": 1024, "hop_length": 128, "n_mels": 64, "center": False},)
    transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=False,
    pad_mode="reflect",
    power=1,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,)
    mel_tensor1 = transform(wav_tensor1).T
    emb_tensor1 = dvector.embed_utterance(mel_tensor1)
    return emb_tensor1
    
def loss_function(AdvTensor, TarTensor):
    return torch.norm(AdvTensor - TarTensor)
def getSign(timestamp, nonce):
    hs = hashlib.sha256()
    appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
    secret = '3c8a61e2bdb0ee2d0af74814142ba2ee'
    hs.update((appkey + timestamp + secret + nonce).encode('utf-8'))
    signature = hs.hexdigest().upper()
    return signature
def identifyFeatureByGroupId(confirmFeatureFileName):
    identify_feature = open(confirmFeatureFileName, 'rb').read()
    # 声纹base64字符串
    audio_data = base64.b64encode(identify_feature)
    timestamp = str(int(time.time() * 1000))
    nonce = str(uuid.uuid1()).replace('-', '')
    sign = getSign(timestamp, nonce)
    headers = {"Content-Type": "application/json"}
    appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
    groupId = '15'
    host = 'https://ai-vpr.hivoice.cn'
    identifyFeatureByGroupIdEndPoint = '/vpr/v1/identifyFeatureByGroupId'
    identify_feature_param = {
        "appkey": appkey,
        "timestamp": timestamp,
        "nonce": nonce,
        "sign": sign,
        "groupId": groupId,
        "topN": 10,
        "audioData": audio_data.decode(),
        "audioSampleRate": 16000,
        "audioFormat": "wav"
    }
    #print('identify_feature_param', identify_feature_param)
    identify_feature_resp = requests.post(url=host + identifyFeatureByGroupIdEndPoint,
                                          data=json.dumps(identify_feature_param),
                                          headers=headers)
    identify_feature_result = json.loads(identify_feature_resp.content)
    return identify_feature_result['data'][0]['featureInfo'], identify_feature_result['data'][0]['score'],identify_feature_result['data'][1]['featureInfo'], identify_feature_result['data'][1]['score']
# for param in wav2mel.parameters():
#     param.requires_grad = False
# for param in dvector.parameters():
#     param.requires_grad = False

In [2]:
path = "/mnt/data/Chenpinji/cmu_dataset"
filename2 = '/mnt/data/Chenpinji/cmu_dataset/cmu7/wav/arctic_a0008.wav'
for cnt in range(4, 200):
    select = random.randint(0,9)
    select2 = random.randint(0,500)
    filename_1 = path + '/' + os.listdir(path)[select] + '/' + 'wav'
    audioname = os.listdir(filename_1)[select2]
    filename1 = filename_1 + '/'+audioname
    wav_tensor1, sample_rate1 = torchaudio.load(filename1, normalize=False)
    wav_tensor2, sample_rate2 = torchaudio.load(filename2, normalize=False)
    noise_tensor = torch.normal(mean = 400., std = 50.,size = (1, len(wav_tensor1[0])))
    wav_tensor1 = wav_tensor1[0].to(torch.float)
    wav_tensor2 = wav_tensor2[0].to(torch.float)
    noise_tensor = noise_tensor[0].to(torch.float)
    #noise_tensor = noise_tensor.cuda()
    noise_tensor.requires_grad = True
    
    
    # wav_tensor1.requires_grad = True
    # wav_tensor2.requires_grad = True
    optimizer = torch.optim.Adam([noise_tensor],lr = 5)
    epochs = random.randint(500,1000)
    for epoch in range(epochs):
        optimizer.zero_grad()
        # AdvTensor = dvector_extraction(wav_tensor1) + dvector_extraction(wav_tensor2)
        # TarTensor = dvector_extraction(noise_tensor)
        #print(TarTensor.grad_fn)
        loss = loss_function(dvector_extraction(wav_tensor1 + noise_tensor), dvector_extraction(wav_tensor2))
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print("epoch={} loss={}".format(epoch, loss))
        if epoch == epochs - 1:
            audio_name = '/mnt/data/Chenpinji/Generate_Adv/dvector_Gen1000/'+'dvectorGen_'+str(cnt) + '.wav'
            temp = wav_tensor1 + noise_tensor
            temp = temp.detach().numpy()
            scaled = np.array(np.clip(np.round(temp),-2 ** 15, 2 ** 15 - 1), dtype = np.int16)
            print("epoch={} loss={}".format(epoch, loss))
            wf.write(audio_name, 16000 ,scaled)
    print(cnt)
        
    

epoch=0 loss=8.67409896850586
epoch=100 loss=3.6485090255737305
epoch=200 loss=3.2117512226104736
epoch=300 loss=2.983950138092041
epoch=400 loss=2.844736337661743
epoch=500 loss=2.729119062423706
epoch=600 loss=2.6353256702423096
epoch=672 loss=2.554264783859253
0
epoch=0 loss=10.836172103881836
epoch=100 loss=7.329015254974365
epoch=200 loss=6.672706127166748
epoch=300 loss=6.089045524597168
epoch=400 loss=5.437991619110107
epoch=500 loss=5.21044921875
epoch=600 loss=5.079563617706299
epoch=606 loss=5.0761027336120605
1
epoch=0 loss=10.570035934448242
epoch=100 loss=7.715263843536377
epoch=200 loss=7.0713300704956055
epoch=300 loss=6.806365013122559
epoch=400 loss=6.725650787353516
epoch=500 loss=6.5749192237854
epoch=600 loss=6.451428413391113
epoch=653 loss=6.3946146965026855
2
epoch=0 loss=10.094398498535156
epoch=100 loss=6.217579364776611
epoch=200 loss=5.447512149810791
epoch=300 loss=4.898906707763672
epoch=400 loss=4.757142066955566
epoch=500 loss=4.633988380432129
epoch=600 

KeyboardInterrupt: 

In [1]:

import torch
torch.cuda.is_available()

True